<a href="https://colab.research.google.com/github/dlagur/ProjectHanium/blob/main/CosineSimilarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## NumPy 이용

In [3]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

def cos_sim(A, B):
    return dot(A, B) / (norm(A)*norm(B))

doc1 = np.array([0,1,1,1])
doc2 = np.array([1,0,2,1])
doc3 = np.array([2,0,2,2])

print('문서 1과 문서 2의 유사도 :', cos_sim(doc1, doc2))
print('문서 2와 문서 3의 유사도 :', cos_sim(doc2, doc3))
print('문서 3과 문서 1의 유사도 :', cos_sim(doc3, doc1))

문서 1과 문서 2의 유사도 : 0.7071067811865476
문서 2와 문서 3의 유사도 : 0.9428090415820635
문서 3과 문서 1의 유사도 : 0.6666666666666667


## 유사도 이용 추천시스템 만들기


In [19]:
pwd = '/content'

In [20]:
pwd

'/content'

In [22]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

data = pd.read_csv('/content/sample_data/movies_metadata.csv', low_memory=False)
data.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [23]:
data = data.head(20000)

In [24]:
print('overview 열의 결측값의 수:', data['overview'].isnull().sum())

overview 열의 결측값의 수: 135


In [25]:
data['overview'] = data['overview'].fillna('') # 결측값을 빈 값으로 대체

In [26]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(data['overview'])
print('TF-IDF 행렬의 크기 :', tfidf_matrix.shape)

TF-IDF 행렬의 크기 : (20000, 47487)


In [27]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
print('코사인 유사도 연산 결과 :', cosine_sim.shape) # (20000, 20000) =  20000개의 각 문서 벡터와 자기 자신 포함 20000개의 문서 벡터 간 유사도 기록

코사인 유사도 연산 결과 : (20000, 20000)


In [30]:
title_to_index = dict(zip(data['title'], data.index))

# 영화 제목 Father of the Bride Part 2의 인덱스를 리턴
idx = title_to_index['Father of the Bride Part II']
print(idx)

4


In [31]:
def get_recommendations(title, cosine_sim = cosine_sim):

    # 선택한 영화의 타이틀로부터 해당 영화의 인덱스를 받아온다
    idx = title_to_index[title]

    # 해당 영화와 모든 영화와의 유사도를 가져온다
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도에 따라 영화들을 정렬한다
    sim_scores = sorted(sim_scores, key=lambda x : x[1], reverse=True)

    #가장 유사한 10개의 영화를 받아온다
    sim_scores = sim_scores[1:11]

    # 가장 유사한 10개 영화의 인덱스를 받아온다
    movie_indices = [idx[0] for idx in sim_scores]

    # 가장 유사한 10개의 영화의 제목을 리턴
    return data['title'].iloc[movie_indices]

In [32]:
get_recommendations('The Dark Knight Rises')

12481                            The Dark Knight
150                               Batman Forever
1328                              Batman Returns
15511                 Batman: Under the Red Hood
585                                       Batman
9230          Batman Beyond: Return of the Joker
18035                           Batman: Year One
19792    Batman: The Dark Knight Returns, Part 1
3095                Batman: Mask of the Phantasm
10122                              Batman Begins
Name: title, dtype: object